In [121]:
import pandas as pd
import dask.dataframe as dd
import os

In [122]:
base_path = "../data/raw/jigsaw-toxix-comments/" 
os.listdir(base_path)

['sample_submission.csv', 'test.csv', 'test_labels.csv', 'train.csv']

In [123]:
train_path = os.path.join(base_path,'train.csv')
print(train_path)
test_path = os.path.join(base_path,'test.csv')
print(test_path)
test_labels_path = os.path.join(base_path,'test_labels.csv')
print(test_labels_path)

../data/raw/jigsaw-toxix-comments/train.csv
../data/raw/jigsaw-toxix-comments/test.csv
../data/raw/jigsaw-toxix-comments/test_labels.csv


In [124]:
df_train = dd.read_csv(train_path)
print(df_train.shape)
df_train.head(2)

(<dask_expr.expr.Scalar: expr=ReadCSV(ec8c79e).size() // 8, dtype=int64>, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [125]:
df_test = dd.read_csv(test_path)
print(df_test.shape)
df_test.head(2)

(<dask_expr.expr.Scalar: expr=ReadCSV(304fe23).size() // 2, dtype=int64>, 2)


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...


In [126]:
df_labels = dd.read_csv(test_labels_path)
df_labels.shape
df_labels.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1


In [127]:
df_labels[(df_labels["toxic"] != -1)].compute().shape

(63978, 7)

In [128]:
df_test = df_test.merge(df_labels, on="id")
df_test.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1


In [129]:
df_test = df_test[df_test.toxic != -1]
df_test.compute().shape

(63978, 8)

In [130]:
cols_for_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df_test[cols_for_labels].sum(axis=1).compute()

0        0
1        0
2        0
3        0
4        0
        ..
63973    0
63974    0
63975    0
63976    3
63977    0
Length: 63978, dtype: int64

In [131]:
df_test.compute().shape

(63978, 8)

In [132]:
df_test["label"] = (df_test[cols_for_labels].sum(axis=1) > 0).astype("int")

In [133]:
df_test["label"].sum().compute()

np.int64(6243)

In [136]:
df_test.rename(columns={"comment_text":"text"}).head()

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0


In [137]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
